In [15]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
import tensorflow as tf
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

#print(os.listdir("/content/input"))

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

# Any results you write to the current directory are saved as output.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [19]:
#lines=pd.read_csv("/content/input/Hindi_English_Truncated_Corpus.csv",encoding='utf-8')
lines = pd.read_csv("https://raw.githubusercontent.com/prismspeechproject/neural/master/implementation/English_hindi.csv" , encoding='utf-8')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7,8,9,10,11,12,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [29]:
lines.rename(columns={'English' : 'english_sentence', 'Hindi': 'hindi_sentence'}, inplace=True)
lines = lines[lines.columns[0:2]]
lines.drop_duplicates(inplace=True)
print(lines.shape)

Before : (124818, 2)
Before : (124818, 2)


In [30]:
pd.isnull(lines).sum()

english_sentence    0  
hindi_sentence      311
dtype: int64

In [32]:
lines=lines[~pd.isnull(lines['english_sentence'])]
lines=lines[~pd.isnull(lines['hindi_sentence'])]
pd.isnull(lines).sum() , lines.shape

(english_sentence    0
 hindi_sentence      0
 dtype: int64, (124507, 2))

In [34]:
lines=lines.sample(n=25000,random_state=42)
lines.shape

(25000, 2)

In [35]:
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.lower())
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.lower())

In [36]:
# Remove quotes
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub("'", '', x))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub("'", '', x))

In [37]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines['english_sentence']=lines['english_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [38]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.translate(remove_digits))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.translate(remove_digits))

lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

# Remove extra spaces
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.strip())
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.strip())
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub(" +", " ", x))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub(" +", " ", x))

In [39]:
# Add start and end tokens to target sequences
lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x : 'START_ '+ x + ' _END')
lines.head()

,english_sentence,hindi_sentence
93994,it is the land of festivals of different religions like diwali dushehra pongal and onam idulfitr muhrram chrimas ester are very much famous,START_ विभिन्न धर्मों के इस भूभाग पर कई मनभावन पर्व त्यौहार मनाए जाते हैं दिवाली होली दशहरा पोंगल तथा ओणम ईदउलफितर मुहर्रम क्रिसमस ईस्टर आदि भी काफ़ी लोकप्रिय हैं। _END
124301,enceladus seen here,START_ एनसेलेडस पर इस जगह _END
28869,ghanaian businessmen south african enterprising leaders,START_ या फिर घाना के व्यवसायियों से या दक्षिण अफ्रिका के उद्योगों में अग्रणी भूमिका निभाने वालों से _END
54938,the burma oil company processed the crude in its refinery at rangoon and distributed the products mainly kerosene through its tank installations at chief indian ports and through a fleet of steam tankers,START_ बर्मा आयल कंपनी अपनी रंगून की रिफाइनरी में कच्चे तेल को साफ करती थी और अपने उत्पादन का वितरण जो मुख़्यत मिट्टी का तेल था प्रधान भारतीय बंदरगाहों पर लगे टैंक प्रतिष्ठानों के द्वारा तथा स्टीम टैंकरों के द्वारा करती थी _END
49997,ismail afandi aka ismail khan of the ottoman empire designer of the main dome,START_ मुख्य गुम्बद का अभिकल्पक इस्माइल एकाइस्माइल खाँ जो कि ऑट्टोमन साम्राज्य का प्रमुख गोलार्ध एवं गुम्बद अभिकल्पक थे। _END


In [40]:
### Get English and Hindi Vocabulary
all_eng_words=set()
for eng in lines['english_sentence']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_hindi_words=set()
for hin in lines['hindi_sentence']:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

In [41]:
len(all_eng_words)

30829

In [42]:
len(all_hindi_words)

37434

In [43]:
lines['length_eng_sentence']=lines['english_sentence'].apply(lambda x:len(x.split(" ")))
lines['length_hin_sentence']=lines['hindi_sentence'].apply(lambda x:len(x.split(" ")))

In [44]:
lines.head()

,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
93994,it is the land of festivals of different religions like diwali dushehra pongal and onam idulfitr muhrram chrimas ester are very much famous,START_ विभिन्न धर्मों के इस भूभाग पर कई मनभावन पर्व त्यौहार मनाए जाते हैं दिवाली होली दशहरा पोंगल तथा ओणम ईदउलफितर मुहर्रम क्रिसमस ईस्टर आदि भी काफ़ी लोकप्रिय हैं। _END,23,30
124301,enceladus seen here,START_ एनसेलेडस पर इस जगह _END,3,6
28869,ghanaian businessmen south african enterprising leaders,START_ या फिर घाना के व्यवसायियों से या दक्षिण अफ्रिका के उद्योगों में अग्रणी भूमिका निभाने वालों से _END,6,19
54938,the burma oil company processed the crude in its refinery at rangoon and distributed the products mainly kerosene through its tank installations at chief indian ports and through a fleet of steam tankers,START_ बर्मा आयल कंपनी अपनी रंगून की रिफाइनरी में कच्चे तेल को साफ करती थी और अपने उत्पादन का वितरण जो मुख़्यत मिट्टी का तेल था प्रधान भारतीय बंदरगाहों पर लगे टैंक प्रतिष्ठानों के द्वारा तथा स्टीम टैंकरों के द्वारा करती थी _END,33,43
49997,ismail afandi aka ismail khan of the ottoman empire designer of the main dome,START_ मुख्य गुम्बद का अभिकल्पक इस्माइल एकाइस्माइल खाँ जो कि ऑट्टोमन साम्राज्य का प्रमुख गोलार्ध एवं गुम्बद अभिकल्पक थे। _END,14,20


In [45]:
lines[lines['length_eng_sentence']>30].shape

(2450, 4)

In [46]:
lines=lines[lines['length_eng_sentence']<=20]
lines=lines[lines['length_hin_sentence']<=20]

In [47]:
lines.shape

(16461, 4)

In [48]:
print("maximum length of Hindi Sentence ",max(lines['length_hin_sentence']))
print("maximum length of English Sentence ",max(lines['length_eng_sentence']))

maximum length of Hindi Sentence  20
maximum length of English Sentence  20


In [49]:
max_length_src=max(lines['length_hin_sentence'])
max_length_tar=max(lines['length_eng_sentence'])

In [50]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
num_encoder_tokens, num_decoder_tokens

(30829, 37434)

In [51]:
num_decoder_tokens += 1 #for zero padding
num_encoder_tokens = num_encoder_tokens + 1 #he forgot about this one he must have put it but deleted it by mistake

In [52]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [53]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [54]:
lines = shuffle(lines)
lines.head(10)

,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
99771,aids is one of those diseases about which we still have much to learn,START_ एड्स एक ऐसी बीमारी है जिसके बारे में हमें अभी बहुत कुछ जानना है _END,14,16
39926,in the tajmahal was declared as unesco world heritage place,START_ में ताजमहल को युनेस्को विश्व धरोहर स्थल घोषित किया गया। _END,10,12
66105,vedwyass g swpretham punycomma including anecdotes of men of a million srlokoan ah made india book,START_ वेदव्यास जी ने सर्वप्रथम पुण्यकर्मा मानवों के उपाख्यानों सहित एक लाख श्लोकों का आद्य भारत ग्रंथ बनाया। _END,16,19
24064,the muslims call this day as pralay,START_ इसे मुसलमान प्रलय का दिन कहते हैं। _END,7,9
106809,and if you ask a three yearold,START_ और अगर आप तीन साल के बच्चे से कहेंगे _END,7,11
122456,and then percent average at,START_ और प्रतिशत पर उस समय _END,5,7
75029,talking about things like gay marriage,START_ समलैंगिक विवाह जैसे विषयों की चर्चा में _END,6,9
104602,now thats a great deal of us isnt it,START_ अब ये कोई बड़ी बात नहीं है _END,9,9
10337,that was run by prostitutes,START_ जिसे वेश्याएँ मिल कर चलाती हैं। _END,5,8
36587,he wrote one article when he was in jail that why he was an atheist,START_ उन्होंने जेल में अंग्रेज़ी में एक लेख भी लिखा जिसका शीर्षक था मैं नास्तिक क्यों हूँ। _END,15,18


In [55]:
X, y = lines['english_sentence'], lines['hindi_sentence']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((13168,), (3293,))

In [56]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')

In [57]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [58]:
latent_dim=300

In [59]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [60]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


In [61]:
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 128

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*batch_size)

# Create a new model instance
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()

train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 30

model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples/batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples/batch_size,callbacks=[cp_callback])

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 300)    9249000     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 300)    11230500    input_2[0][0]                    
______________________________________________________________________________________________

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
102/102 [==============================] - 85s 475ms/step - loss: 4.0930 - val_loss: 3.4967
Epoch 2/30
102/102 [==============================] - 45s 441ms/step - loss: 3.4893 - val_loss: 3.4316
Epoch 3/30
102/102 [==============================] - 45s 442ms/step - loss: 3.3501 - val_loss: 3.3342
Epoch 4/30
102/102 [==============================] - 44s 428ms/step - loss: 3.2105 - val_loss: 3.2734
Epoch 5/30
102/102 [==============================] - 44s 431ms/step - loss: 3.0929 - val_loss: 3.2345
Epoch 6/30
102/102 [==============================] - 45s 434ms/step - loss: 2.9900 - val_loss: 3.2246
Epoch 7/30
 22/102 [=====>........................] - ETA: 30s - loss: 2.9506
Epoch 00007: saving model to training_2/cp-0007.ckpt
102/102 [==============================] - 45s 442ms/step - loss: 2.8969 - val_loss: 3.1836
Epoch 8/30
102/102 [==============================] - 44s 430ms/step - loss: 2.8081 - val_loss: 3.1744
Epoch 9/30
102/102 [==============================] - 45

In [62]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [66]:
model.save_weights('/content/drive/MyDrive/ColabNotebooks/nmt_weights21.h5')
model.save('/content/drive/MyDrive/ColabNotebooks/nmt_weights21save.h5')

In [67]:
from keras.models import load_model
new_model = load_model('/content/drive/MyDrive/ColabNotebooks/nmt_weights21save.h5')

# Show the model architecture
new_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 300)    9249000     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 300)    11230500    input_2[0][0]                    
______________________________________________________________________________________________

In [68]:
#new_model = load_model(filepath)'

os.listdir(checkpoint_dir)

['cp-0000.ckpt.data-00000-of-00001',
 'cp-0007.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.index',
 'checkpoint',
 'cp-0007.ckpt.index',
 'cp-0019.ckpt.index',
 'cp-0025.ckpt.index',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0019.ckpt.data-00000-of-00001',
 'cp-0013.ckpt.index',
 'cp-0013.ckpt.data-00000-of-00001']

In [69]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0025.ckpt'

In [97]:
# X_test = X_test.apply(lambda x: str(x))
# y_test = y_test.apply(lambda x: str(x))
X_test = X_test.astype(str)
y_test = y_test.astype(str)
X_test[:1][0]

KeyError: ignored

In [89]:
#nmodel = create_model()
#nmodel = Model([encoder_inputs, decoder_inputs], decoder_outputs)
# Load the previously saved weights
#nmodel.load_weights(latest)
print(X_test.dtypes , X_train.dtypes, y_test.dtypes, y_train.dtypes)
loss, acc = model.evaluate([X_test, y_test], batch_size=128)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

object object object object


UnimplementedError: ignored

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
new_model = tf.keras.models.load_model('/content/nmt_weights.h5')

# Show the model architecture
new_model.summary()

TypeError: ignored

In [ ]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: thats the first time in india
Actual Hindi Translation:  भारत में ये पहली बार हुआ 
Predicted Hindi Translation:  भारत में यह पहली बार हुआ 


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: i was waiting and when he did emerge
Actual Hindi Translation:  मैं इन्तेज़ार कर रहा था और जब वो आया 
Predicted Hindi Translation:  मैं कैसे मार्गदर्शन जब मैं कहना गया था 


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: whod never been interested in art
Actual Hindi Translation:  जिसे कला में बिल्कुल भी रूचि नहीं थी 
Predicted Hindi Translation:  जिसे कला में कभी भी ऐसा नहीं थी 


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: it saves lives
Actual Hindi Translation:  यह जीवन बचाता है 
Predicted Hindi Translation:  यह जीवन 
